In [32]:
import time
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

In [33]:
#Mail an eine Mail Adresse senden
#https://stackoverflow.com/questions/23171140/how-do-i-send-an-email-with-a-csv-attachment-using-python
def sendmail(path):
    emailfrom = "FROM:EMAIL"
    emailto = "TO:EMAIL" 
    fileToSend = path
    username = "USER"
    password = "PW"

    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = emailto
    msg["Subject"] = "Tweets-Alert!"
    msg.preamble = "Tweets-Alert!"

    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(fileToSend, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "audio":
        fp = open(fileToSend, "rb")
        attachment = MIMEAudio(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    server.sendmail(emailfrom, emailto, msg.as_string())
    server.quit()

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("SSKafka") \
    .getOrCreate()
    
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
tweets = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "13.74.113.150:9092") \
  .option("subscribe", "benten123") \
  .option("startingOffsets", "earliest") \
  .load()

In [35]:
tweets.isStreaming

True

In [36]:
tweets = tweets.selectExpr("timestamp", "CAST(value AS STRING)")

In [37]:
tweets.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: string (nullable = true)



In [38]:
counting_8=tweets.withWatermark("timestamp", "2 minutes").groupBy("value", window("timestamp", "1 minutes")).count().where("count > 5")

In [39]:
counting_8=counting_8.selectExpr("value", "CAST(window AS STRING)", "count")

In [41]:
TweetsQuery = counting_8 \
        .writeStream \
        .queryName("tweetsYXXX")\
        .format("memory")\
        .start()

22/01/19 18:31:40 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-02be0977-caa2-4945-946b-b3dc79c416c0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [ ]:
#tweets_stream2 = spark.sql("select * from tweets2 order by timestamp(substring(window,2,16)), value desc")
#tweets_stream2.show()

In [ ]:
tweets_stream = spark.sql("select * from tweetsYXXX where timestamp(substring(window,2,16)) > (current_timestamp() - INTERVAL 10 minutes) order by window, value")
tweets_stream.show()


In [69]:
#send Mail Alerts if count > 5
while counting_8.isStreaming:
    if tweets_stream.rdd.isEmpty() == True:
        time.sleep(5)
    elif tweets_stream.rdd.isEmpty() == False:
        tweets_stream.toPandas().to_csv('Desktop/data_alias.csv', header = True)
        tweets_stream.show()
        sendmail('Desktop/data_alias.csv')#
        time.sleep(60)
    

KeyboardInterrupt: 